In [9]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import pandas as pd
import time
from bs4 import BeautifulSoup

# Set up Chrome options
chrome_options = Options()
chrome_options.add_argument("--start-minimized")
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

# Set up the Chrome driver
chromedriver_path = '/opt/homebrew/bin/chromedriver'  # Modify this path
service = Service(chromedriver_path)

# Initialize Chrome WebDriver
driver = webdriver.Chrome(service=service, options=chrome_options)

# Initialize DataFrame
job_dataset = pd.DataFrame(columns=["Job Title", "Company", "Location", "Description"])

url = 'https://example.com'  # Replace with your URL

try:
    driver.get(url)
    time.sleep(5)  # Wait for the page to load

    # Scroll to the bottom of the page to load all job listings
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    # Parse the page source using BeautifulSoup
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    job_cards = soup.find_all('div', class_='job-search-card')

    for job_card in job_cards:
        try:
            job_title = job_card.find('h3', class_='job-title').get_text(strip=True)
            company = job_card.find('h4', class_='company-name').get_text(strip=True)
            location = job_card.find('span', class_='job-location').get_text(strip=True)
            job_link = job_card.find('a', class_='job-link')['href']

            # Navigate to the job description page
            driver.get(job_link)
            time.sleep(2)
            job_desc_soup = BeautifulSoup(driver.page_source, 'html.parser')
            job_description = job_desc_soup.find('div', class_='job-description').get_text(strip=True)

            # Append data to DataFrame
            job_dataset = job_dataset.append({
                "Job Title": job_title,
                "Company": company,
                "Location": location,
                "Description": job_description
            }, ignore_index=True)

        except Exception as e:
            print(f"Error parsing job card: {e}")

finally:
    driver.quit()

# Save the DataFrame to a CSV file
job_dataset.to_csv("data_science_jobs_linkedin.csv", index=False)



TypeError: WebDriver.__init__() got an unexpected keyword argument 'service'